#### 一、基本用法
1. `awk`的基本用法就是下面的形式。

    ```bash
    # 格式
    $ awk 脚本片段 文件名

    # 示例
    $ awk '{print $0}' demo.txt
    ```
  
  上面示例中，demo.txt是awk所要处理的文本文件。前面单引号内部有一个大括号，里面就是每一行的处理动作print \\$0  
  其中，print是打印命令，$0代表当前行，因此上面命令的执行结果，就是把每一行原样打印出来。
  
  
2. awk对每一行划分为不同字段  
    * awk会根据`空格`和`制表符`，将每一行分成若干字段，依次用\\$1、\\$2、\\$3代表第一个字段、第二个字段、第三个字段等等。
    ```bash
    $ echo "this is a cat" | awk '{print $1}'
    this
    ```
   上面代码中，\\$1代表this is a test的第1个字段this。
   
   * 通过`-F`制定分隔符(代码片段和分隔符都要用单引号包括)  
     首文件中内容如下  
    ```
    root:x:0:0:root:/root:/usr/bin/zsh
    daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
    bin:x:2:2:bin:/bin:/usr/sbin/nologin
    sys:x:3:3:sys:/dev:/usr/sbin/nologin
    sync:x:4:65534:sync:/bin:/bin/sync
    ```
    awk处理文件
    ```bash
    $ awk -F ':' '{print $1}' demo.txt 
    root
    daemon
    bin
    sys
    sync
   ```
   
#### 二、变量  
1. 除了`$ + 数字`表示某个字段，awk还提供其他一些变量。   
    * `NF`表示当前行的字段个数
    * `$NF`表示最后一个字段  
    * `$(NF-1)` 表示倒数第二个字段
    ```bash
    $ awk -F ':' '{print $(NF-1),$NF}' demo.txt 
   /root , /usr/bin/zsh
   /usr/sbin , /usr/sbin/nologin
   /bin , /usr/sbin/nologin
   /dev , /usr/sbin/nologin
   /bin , /bin/sync
    ```  
    上面的print的输出里, 字段之间默认使用`空格`隔开, 要想在print命令里面原样输出字符，要放在双引号里面。
    * `NR`: 表示现在是第几行  
    ```bash
    $ awk -F ':' '{print NR")"$1}' demo.txt 
    1)root
    2)daemon
    3)bin
    4)sys
    5)sync
    ```

2. awk的其他内置变量如下。
    * FILENAME：当前文件名
    * FS：字段分隔符，默认是空格和制表符。
    * RS：行分隔符，用于分割每一行，默认是换行符。
    * OFS：输出字段的分隔符，用于打印时分隔字段，默认为空格。
    * ORS：输出记录的分隔符，用于打印时分隔记录，默认为换行符。
    * OFMT：数字输出的格式，默认为％.6g。
    
    
#### 三. 条件
1. `awk`允许指定输出条件，只输出符合条件的行。   
    * 输出条件要写在动作的前面。
   ```bash
   $ awk '条件 动作' 文件名
   ```
    * 下面代码中，`print`命令前面是一个正则表达式，只输出包含usr的行。
    
    ```bash
    $ awk -F ':' '/usr/ {print $1}' demo.txt
    root
    daemon
    bin
    sys
    ```
    
    * 下面的例子只输出奇数行，以及输出第三行以后的行。
    ```bash
    # 输出奇数行
    $ awk -F ':' 'NR % 2 == 1 {print $1}' demo.txt
    root
    bin
    sync
    ```
    
    ```bash
    # 输出第三行以后的行
    $ awk -F ':' 'NR >3 {print $1}' demo.txt
    sys
    sync
    ```
    
2. `if-else`语句
    * awk提供了if结构，用于编写复杂的条件。   
    下面代码输出第一个字段的第一个字符大于m的行。   
    ```bash
    $ awk -F ':' '{if ($1 > "m") print $1}' demo.txt
       root
       sys
       sync
    ```

    * if结构还可以指定else部分。  

    ```bash     
    $ awk -F ':' '{if ($1 > "m") print $1; else print "---"}' demo.txt
    ```

#### 三. BEGIN语句块
1. 使用BEGIN语句块定义表头
```bash
$ awk 'BEGIN{print "姓名 语文 数学 英语"}{printf "%-8s%-5d%-5d%-5d\n",$1,$2,$3,$4}' score.txt
   姓名 语文 数学 英语
   tom     60   60   60   
   kitty   90   95   87   
   jack    72   84   99  
```

2. 使用BEGIN语句块初始化变量
```shell
39  246  707
[@tc_59_134 lj]$ awk '
> BEGIN{
>     print "姓名 语文 数学 英语 总成绩";
>     sum1=0;sum2=0;sum3=0;sumall=0;}
> {
>     printf "%5s%5d%5d%5d%5d\n",$1,$2,$3,$4,$2+$3+$4;
>     sum1+=$2;sum2+=$3;sum3+=$4;sumall+=$2+$3+$4;
> }
> 
> END{
>     printf "%5s%5d%5d%5d%5d\n","总成绩",sum1,sum2,sum3,sumall;
> }
> ' score.txt
姓名 语文 数学 英语 总成绩
  tom   60   60   60  180
kitty   90   95   87  272
 jack   72   84   99  255
总成绩  222  239  246  707
```

3. 定义函数代码块
```shell
$ awk 'BEGIN{print "姓名 语文 数学 英语 总成绩"; \
> sum1=0;sum2=0;sum3=0;sumall=0} \
> function addAll(yuwen,shuxue,yingyu){ \
>     return yuwen+shuxue+yingyu; \
> } \
> {printf "%5s%5d%5d%5d%5d\n",$1,$2,$3,$4,$2+$3+$4; \
> sum1+=$2;sum2+=$3;sum3+=$4;sumall+=$2+$3+$4} \
> END{printf "%5s%5d%5d%5d%5d\n","总成绩",sum1,sum2,sum3,sumall}' \
>  score.txt
姓名 语文 数学 英语 总成绩
  tom   60   60   60  180
kitty   90   95   87  272
 jack   72   84   99  255
总成绩  222  239  246  707
```

4. awk进行分组聚合
```bash
awk -F"\t" '
{
    key = $1;
    bill_click[key] += $2;
    cd_click[key] += $3;
    cost[key] += $4;
} END {
    for (ele in bill_click) {
        printf("%s\t%s\t%s\t%s\n", ele, bill_click[ele], cd_click[ele], cost[ele]);
    }
}
' ./tmp/storm_bill_cd > ./tmp/merged_bill_cd
```